In [8]:
import praw
import pandas as pd
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# ---------------- Reddit API Setup ----------------
reddit = praw.Reddit(
    client_id="sXO_IEeGyvCl5Hx7aou66w",  # from the top of the image
    client_secret="G933fpDEDWGgses_PoLmfVDrtD9uCQ",  # the 'secret' field
    username="Green_Taro_748 ",  # your Reddit username
    password="Dlwodnjs13!",  # your actual Reddit login password
    user_agent="script:ev.sentiment.analyzer:v1.0 (by u/Abject_Praline3611)"
)

# ---------------- Sentiment Analyzer Setup ----------------
analyzer = SentimentIntensityAnalyzer()

# ---------------- Define Brands & Queries ----------------
brands = [
    {"query": "Cybertruck", "label": "Tesla Cybertruck"},
    {"query": "Ford F-150 Lightning", "label": "Ford F-150 Lightning"},
    {"query": "Rivian R1T", "label": "Rivian R1T"},
    {"query": "GMC Hummer EV", "label": "GMC Hummer EV"},
    {"query": "Chevy Silverado EV", "label": "Chevy Silverado EV"}
]

# ---------------- Extract Posts + Comments ----------------
all_data = []

for brand in brands:
    print(f"\n🔍 Searching posts for: {brand['label']}")
    posts = reddit.subreddit("electricvehicles+cars+teslamotors+F150Lightning+Rivian").search(
        brand["query"], limit=40)

    for post in posts:
        post_date = datetime.fromtimestamp(post.created_utc).date()
        sentiment = analyzer.polarity_scores(post.title)

        all_data.append({
            'post_or_comment': 'Post',
            'product_label': brand['label'],
            'content': post.title,
            'score': post.score,
            'num_comments': post.num_comments,
            'created_date': post_date,
            'sentiment_neg': sentiment['neg'],
            'sentiment_pos': sentiment['pos'],
            'sentiment_compound': sentiment['compound']
        })

        # Extract and analyze comments
        post.comments.replace_more(limit=0)
        for comment in post.comments:
            comment_date = datetime.fromtimestamp(comment.created_utc).date()
            comment_sentiment = analyzer.polarity_scores(comment.body)

            all_data.append({
                'post_or_comment': 'Comment',
                'product_label': brand['label'],
                'content': comment.body,
                'score': comment.score,
                'num_comments': None,
                'created_date': comment_date,
                'sentiment_neg': comment_sentiment['neg'],
                'sentiment_pos': comment_sentiment['pos'],
                'sentiment_compound': comment_sentiment['compound']
            })

# ---------------- Save to CSV ----------------
df = pd.DataFrame(all_data)
df.to_csv("reddit_api_data.csv", index=False)


🔍 Searching posts for: Tesla Cybertruck

🔍 Searching posts for: Ford F-150 Lightning

🔍 Searching posts for: Rivian R1T

🔍 Searching posts for: GMC Hummer EV

🔍 Searching posts for: Chevy Silverado EV
Sentiment data saved to 'reddit_sentiment_tesla_and_competitors.csv'
